<a href="https://colab.research.google.com/github/gbardaklis/Mini-Project-2/blob/main/MP2_lina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

order of importance, 
car, 
board, 
puzzle parser

In [4]:
# imports 
import copy
import numpy as np

# Position Class 

In [ ]:
class Position():
  def __init__(self, posX, posY):
    self.posX = posX
    self.posY = posY
  
  def __str__(self):
    return f"({str(self.posX)},{str(self.posY)})" 

# Car Class 

In [83]:
class Car(object):
  def __init__(self, name, position, fuel = 100, dir = "v"):
    self.name = name
    self.fuel = int(fuel)
    self.dir = dir
    self.positions = []
    self.positions.append(position)

  def getName(self) -> str:
    return self.name; 
    
  def isAmbulance(self) -> bool:
    return self.name == 'A'

  def isName(self, name) -> bool:
    return self.name == name

  # check if car has enough fuel to move, returns boolean
  def canMove(self) -> bool:
    return self.fuel > 0

  # may need to be changed
  def moveCarByLength(self, length):
    if (self.direction == (dir == "h")):
      for car in self.positions:
        car.posX += length
    else:
      for car in self.positions:
        car.posY += length
    self.fuel -= length

  def updatePosition(self, updatePosition):
    self.positions.append(updatePosition)
    if self.positions[0].posX == updatePosition.posX and self.positions[0].posY != updatePosition.posY:
      self.direction = "h"
    else:
      self.direction = "v"

  def __str__(self) -> str:
    x = self.name + ' | '
    for iter in self.positions: 
      x += str(iter)
    return f" {x} | {self.fuel} | {self.direction}"

# Board Class 

In [69]:
class Board(object): 
  def __init__(self, cars):
    self.cars = []
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    for car in cars:
      carHolder = car
      self.cars.append(copy.deepcopy(carHolder))
      # updaying grid with the coordinate of the cars
      for position in carHolder.positions:
        # assign the letter at that position
        self.grid[position.posX][position.posY] = car.name

  def win(self):
    if self.grid[2][5] == 'A':
      return True
    else:
      return False

  def pathForward(self, car):
    direction = car.direction
    position = car.positions[len(car.positions)-1]
    if direction == 'h':
      # may need to be swapped to posY, posX
      head = position.posX
    else:
      head = position.posY
    
    moves = []
    for iter in range(head+1, 6):
      if car.fuel < iter-head:
        continue
      if direction == 'h':
        gridPosition = self.grid[position.posX][iter]
      else:
        gridPosition = self.grid[iter][position.posY]
      if gridPosition == '.':
        moves.append(iter-head)
      else:
        break
    return moves

  def pathBackward(self, car):
      direction = car.direction
      position = car.positions[0]
      if direction == 'h':
        head = position.posX
      else:
        head = position.posY
      
      moves = []
      for iter in range(head, 0, -1):
        if car.fuel <= head-iter:
          continue
        if direction == 'h':
          gridPosition = self.grid[position.posX][iter-1]
        else:
          gridPosition = self.grid[iter-1][position.posY]
        if gridPosition == '.':
          moves.append(iter-head-1)
        else:
          break
      return moves

  def findCarMoves(self, car):
    if car.canMove():
      forward = self.pathForward(car)
      backward = self.pathBackward(car)
      path = forward + backward
    else: 
      path = []
    return path

  def moveCar(self, name, length):
    for car in self.cars:
      if car.name == name:
        for position in car.positions:
          self.grid[position.posX][position.posY] = '.'
        car.moveCarByLength(length)
        for position in car.positions:
          self.grid[position.posX][position.posY] = name

        # car != ambulance reached exit, coord (2,5) h direction, game needs to continue
        if car.name == self.grid[2][5] and car.dir =='h' and car.name != 'A':
          for position in car.positions: 
            self.grid[position.posX][position.posY] = '.'
          self.cars.remove(car)

        direction = ""
        if length < 0 and car.direction == 'h':
          direction = 'left'
        if length > 0 and car.direction == 'h':
          direction = 'right'
        if length < 0 and car.direction == 'v':
          direction = 'down'
        if length > 0 and car.direction == 'v':
          direction = 'up'
        return name, direction, length, car.fuel, self.puzzle()

  # populate grid
  def puzzle(self):
    puzzle = ""
    for x in range(6):
      for y in range(6):
        puzzle += self.grid[x][y]
    return puzzle

  # grid default to string
  def __str__(self): 
    puzzle = ""
    for x in range(6):
      for y in range(6):
        puzzle += self.grid[x][y]
      puzzle += "\n" 
    return puzzle

# PuzzleParser Class 

In [94]:
class PuzzleParser:
  def createPuzzleList():
      input = open('/content/sample-input.txt', 'r')
      puzzles = []
      for line in input.readlines():
        if not line.startswith('#') and line.strip():
            puzzles.append(line.split(" ")) 
      input.close()
      return puzzles

# Testing Game Set Up

In [ ]:
puzzler = PuzzleParser
puzzleMap = []
puzzles = puzzler.createPuzzleList();

for line in puzzles:
  grid = line[0];
  fuelLevel = []
  if len(line) > 1:
    fuelLevel.append(line[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      if current == ".":
        continue
      isSame = True
      for car in cars:
        if car.isName(current):
          car.updatePosition(Position(row, col))
          isSame = False
      if isSame:
        fuelAmt = 100
        if len(fuelLevel) > 0:
          for fuel in fuelLevel[0]:
            if current == fuel[0]:
              fuelAmt = fuel[1:]

        cars.append(Car (name = current, position = Position(row,col), fuel = fuelAmt))

  board = Board(cars)
  print(board)
  for car in cars:
    print(car)
  print("---------------------------------")
  puzzleMap.append(board)

# LIST OF UNUSED FUCTIONS ❗ 💯

This function returns a list of cars with their FUEL from the list of puzzles

In [11]:
# this should become a PuzzleParser Class 
def __create_puzzle_list():
    input = open('/content/sample-input-2.txt', 'r')
    puzzles = []
    for line in input.readlines():
      if not line.startswith('#') and line.strip():
          puzzles.append(line.strip()) 
    input.close()
    return puzzles
  
  # seperates the fuel from the board 
def __get_fuel():
    # a list of cars temp
    carsTemp = []
    # a list of car objects
    cars = []
    for x in puzzles:
      puzzle_str = x.split(" ")
      # if only 0 then theres only a puzzle, default 100
      # else fuel assigned
      if len(puzzle_str) > 0:
        for i in range (1, len(puzzle_str)):
          carsTemp.append(puzzle_str[i])
          carSample = carsTemp[i-1]
          carSampleLetter = carSample[0]
          carSampleFuel = carSample[1]
          cars.append(Car (name = carSampleLetter, fuel = carSampleFuel))
        return cars

In [12]:
puzzles = __create_puzzle_list();

In [ ]:
a = __get_fuel()
print(a[0].name)

Writes to a file the desired layout with the fuel and car name

In [ ]:
# def __output():
output = open("/content/test.txt", "w")
for x in puzzles:
  output.write(f"Initial board configuration: {x} \n\n")
  splitted_line = x.split(" ")
  puzzlestr = splitted_line[0]
  count = 0
  for i in range(len(puzzlestr)):
    output.write(puzzlestr[i] + " ")
    count += 1
    if count == 6:
      output.write("\n")
      count = 0
  output.write("\n")
  output.write("Car fuel available: ")
  for f in a: 
    output.write(f"{f.name}:{f.fuel}, ")

This takes in a puzzle string and counts the occurrences of the char to make the SIZE of the car

In [ ]:
def __checkSize(puzzle): 
  cars = []
  # Test case
  # input_string = "BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL."
  mySet = set(puzzle)
  countOfChars = dict()
  for element in mySet:
      countOfChar = puzzle.count(element)
      countOfChars[element] = countOfChar
      if (element == "."):
        continue;
      else:  
        cars.append(Car (name = element, size = countOfChar))

  counter = 0;
  for x in cars:
    print (cars[counter].name + str(cars[counter].size) ) 
    counter += 1;  
  return cars

Checks if a car is setup HORIZONTALLY

In [ ]:
def __checkHorizontal(puzzle): 
  uniqueHorizontal = []
  for curr,next in zip(puzzle, puzzle[1:]): 
    if (curr == "."):
      continue;
    if (curr == next) :
      uniqueHorizontal.append(curr)
    else:
      continue;
  finalHorizontal = set(uniqueHorizontal)    
  return finalHorizontal

This is the Check for Horizontal and Size Functions

In [ ]:
newString = puzzles[0]
newString = newString[:36]
returnedHorizontalCar = __checkHorizontal(newString)
print(returnedHorizontalCar)

# print('\n' + 'this is the checksize function')
returnedCarSize = __checkSize(newString)
print(returnedCarSize)

TO REVISIT 

In [ ]:
#Combining Both lists
counter2=0
for (currentCar, horizontalCar) in zip(returnedCarSize, returnedHorizontalCar):

  thisIsTheCar = returnedCarSize[counter2].name
  if ( thisIsTheCar != horizontalCar):
    print("this is not a match" + thisIsTheCar + horizontalCar)
  if ( thisIsTheCar == horizontalCar):
    returnedCarSize[counter2].dir = "h"
    counter2+=1  
  else:
    counter2+=1


counter = 0;
for x in returnedCarSize:
    print (returnedCarSize[counter].name + str(returnedCarSize[counter].size) + returnedCarSize[counter].dir) 
    counter += 1;  
# print(returnedCarSize[0].dir) 


Slices the puzzle string into 36 chars (6x6) passes into ndarray 

In [ ]:
newString = puzzles[0]
newString = newString[:36]
print(newString)

test = np.array(list(newString)).reshape(6,6)
print(test)
print(test[0][4])
print(type(test))



BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII
[['B' 'B' 'B' '.' '.' 'M']
 ['C' 'C' 'D' 'D' '.' 'M']
 ['A' 'A' 'K' 'L' '.' 'M']
 ['J' '.' 'K' 'L' 'E' 'E']
 ['J' '.' 'G' 'G' '.' '.']
 ['J' 'H' 'H' 'H' 'I' 'I']]
.
<class 'numpy.ndarray'>


In [ ]:
board1 = Board(board = "BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII")
print(board1.board)

BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII


In [ ]:
board2 = Board(board = puzzles[0])
print(board2.board)

BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.


In [ ]:
# def __play_rush_hour_ucs():
#   # returns list of puzzles 
#   puzzles = __create_puzzle_list();
#   for index, input in enumerate(puzzles):
#     __puzzle_output_ucs(input, f"ucs-sol-{index+1}.txt")


In [ ]:
# NOT GOING TO BE USED 
# Makes a DICT of occurrences and chars
test = "BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL."

letterDict = dict()

# print(letterDict)

letterList = []

for x in test:
  occurences = test.count(x)
  # print(occurences)
  if (x == ".") or (x in letterList):
    continue; 
  else:
    letterList.append(f"{x} + {occurences}")
    
finalList = set(letterList)
print(finalList)

{'G + 3', 'A + 2', 'D + 2', 'I + 3', 'C + 2', 'H + 2', 'F + 2', 'B + 2', 'J + 2', 'K + 2', 'M + 2', 'L + 2'}
